<a href="https://colab.research.google.com/github/henatic/Machine-Learning-with-Python_freeCodeCamp/blob/main/Copy_of_fcc_sms_text_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# import libraries
try:
  # %tensorflow_version only exists in Colab.
  !pip install tf-nightly
except Exception:
  pass
import tensorflow as tf
import pandas as pd
from tensorflow import keras
!pip install tensorflow-datasets
import tensorflow_datasets as tfds
import numpy as np
import matplotlib.pyplot as plt

print(tf.__version__)

2.18.0-dev20240607


In [3]:
# get data files
!wget https://cdn.freecodecamp.org/project-data/sms/train-data.tsv
!wget https://cdn.freecodecamp.org/project-data/sms/valid-data.tsv

train_file_path = "train-data.tsv"
test_file_path = "valid-data.tsv"

--2024-06-07 22:44:39--  https://cdn.freecodecamp.org/project-data/sms/train-data.tsv
Resolving cdn.freecodecamp.org (cdn.freecodecamp.org)... 172.67.70.149, 104.26.2.33, 104.26.3.33, ...
Connecting to cdn.freecodecamp.org (cdn.freecodecamp.org)|172.67.70.149|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 358233 (350K) [text/tab-separated-values]
Saving to: ‘train-data.tsv’

train-data.tsv      100%[===================>] 349.84K  --.-KB/s    in 0.02s   

2024-06-07 22:44:39 (20.2 MB/s) - ‘train-data.tsv’ saved [358233/358233]

--2024-06-07 22:44:39--  https://cdn.freecodecamp.org/project-data/sms/valid-data.tsv
Resolving cdn.freecodecamp.org (cdn.freecodecamp.org)... 172.67.70.149, 104.26.2.33, 104.26.3.33, ...
Connecting to cdn.freecodecamp.org (cdn.freecodecamp.org)|172.67.70.149|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 118774 (116K) [text/tab-separated-values]
Saving to: ‘valid-data.tsv’

valid-data.tsv      100%[==========

In [4]:
myNames = ["class", "message"]

In [5]:
train_file = pd.read_csv(train_file_path, sep='\t', names=myNames)
train_file

,class,message
0,ham,ahhhh...just woken up!had a bad dream about u ...
1,ham,you can never do nothing
2,ham,"now u sound like manky scouse boy steve,like! ..."
3,ham,mum say we wan to go then go... then she can s...
4,ham,never y lei... i v lazy... got wat? dat day ü ...
...,...,...
4174,ham,just woke up. yeesh its late. but i didn't fal...
4175,ham,what do u reckon as need 2 arrange transport i...
4176,spam,free entry into our £250 weekly competition ju...
4177,spam,-pls stop bootydelious (32/f) is inviting you ...


In [6]:
test_file = pd.read_csv(test_file_path, sep='\t', names=myNames)
test_file

,class,message
0,ham,i am in hospital da. . i will return home in e...
1,ham,"not much, just some textin'. how bout you?"
2,ham,i probably won't eat at all today. i think i'm...
3,ham,don‘t give a flying monkeys wot they think and...
4,ham,who are you seeing?
...,...,...
1387,ham,true dear..i sat to pray evening and felt so.s...
1388,ham,"what will we do in the shower, baby?"
1389,ham,where are you ? what are you doing ? are yuou ...
1390,spam,ur cash-balance is currently 500 pounds - to m...


In [7]:
train_list = train_file["message"].values.tolist()
train_label = np.array([0 if x=="ham" else 1 for x in train_file['class'].values.tolist()])

test_list = train_file["message"].values.tolist()
test_label = np.array([0 if x=="ham" else 1 for x in train_file['class'].values.tolist()])

In [8]:
vocab = {}
for message in train_list:
  for vocabulary in message.split():
    if vocabulary not in vocab:
      vocab[vocabulary] = 1
    else:
      vocab[vocabulary] += 1

VOCABULARY_SIZE = len(vocab)
MAX_LENGTH = len(max(train_list, key=lambda p: len(p.split())).split())

In [9]:
train_message_encoded = [keras.preprocessing.text.one_hot(d, VOCABULARY_SIZE) for d in train_list]
train_message_padded = keras.preprocessing.sequence.pad_sequences(train_message_encoded, maxlen=MAX_LENGTH, padding='post')

test_message_encoded = [keras.preprocessing.text.one_hot(d, VOCABULARY_SIZE) for d in test_list]
test_message_padded = keras.preprocessing.sequence.pad_sequences(test_message_encoded, maxlen=MAX_LENGTH, padding='post')

In [10]:
model = keras.models.Sequential()
embed_layer = keras.layers.Embedding(VOCABULARY_SIZE, 100, input_length=MAX_LENGTH)
model.add(embed_layer)
model.add(keras.layers.Flatten())
model.add(keras.layers.Dense(1, activation='sigmoid'))
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])
monitor = keras.callbacks.EarlyStopping(monitor='val_acc', min_delta=1e-4, patience=25, verbose=1, mode='max', restore_best_weights=True)
model.fit(train_message_padded, train_label, validation_data=(test_message_padded, test_label), callbacks=[monitor], epochs=1000, verbose=2)

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Epoch 1/1000
131/131 - 4s - 29ms/step - acc: 0.9210 - loss: 0.2308 - val_acc: 0.9785 - val_loss: 0.0792
Epoch 2/1000
131/131 - 3s - 23ms/step - acc: 0.9842 - loss: 0.0568 - val_acc: 0.9895 - val_loss: 0.0342
Epoch 3/1000
131/131 - 2s - 19ms/step - acc: 0.9923 - loss: 0.0287 - val_acc: 0.9966 - val_loss: 0.0171
Epoch 4/1000
131/131 - 2s - 15ms/step - acc: 0.9957 - loss: 0.0151 - val_acc: 0.9990 - val_loss: 0.0085
Epoch 5/1000
131/131 - 2s - 15ms/step - acc: 0.9988 - loss: 0.0080 - val_acc: 0.9998 - val_loss: 0.0051
Epoch 6/1000
131/131 - 3s - 20ms/step - acc: 0.9995 - loss: 0.0051 - val_acc: 0.9998 - val_loss: 0.0039
Epoch 7/1000
131/131 - 2s - 16ms/step - acc: 0.9998 - loss: 0.0037 - val_acc: 0.9998 - val_loss: 0.0027
Epoch 8/1000
131/131 - 3s - 23ms/step - acc: 0.9998 - loss: 0.0028 - val_acc: 0.9998 - val_loss: 0.0021
Epoch 9/1000
131/131 - 2s - 16ms/step - acc: 0.9998 - loss: 0.0022 - val_acc: 0.9998 - val_loss: 0.0018
Epoch 10/1000
131/131 - 3s - 19ms/step - acc: 0.9998 - loss: 0.0

In [11]:
# function to predict messages based on model
# (should return list containing prediction and label, ex. [0.008318834938108921, 'ham'])
def predict_message(pred_text):
  classes = {
      0: "ham",
      1: "spam"
  }

  encoded_message = [keras.preprocessing.text.one_hot(pred_text, VOCABULARY_SIZE)]
  padded_message = keras.preprocessing.sequence.pad_sequences(encoded_message, maxlen=MAX_LENGTH, padding='post')
  prediction = [model.predict(padded_message)[0][0], classes[np.round(model.predict(padded_message)[0][0])]]


  return (prediction)

pred_text = "how are you doing today?"

prediction = predict_message(pred_text)
print(prediction)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 118ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
[0.0006282649, 'ham']


In [12]:
# Run this cell to test your function and model. Do not modify contents.
def test_predictions():
  test_messages = ["how are you doing today",
                   "sale today! to stop texts call 98912460324",
                   "i dont want to go. can we try it a different day? available sat",
                   "our new mobile video service is live. just install on your phone to start watching.",
                   "you have won £1000 cash! call to claim your prize.",
                   "i'll bring it tomorrow. don't forget the milk.",
                   "wow, is your arm alright. that happened to me one time too"
                  ]

  test_answers = ["ham", "spam", "ham", "spam", "spam", "ham", "ham"]
  passed = True

  for msg, ans in zip(test_messages, test_answers):
    prediction = predict_message(msg)
    if prediction[1] != ans:
      passed = False

  if passed:
    print("You passed the challenge. Great job!")
  else:
    print("You haven't passed yet. Keep trying.")

test_predictions()


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
You passed the challenge. Great job!
